# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [4]:
encoded[:100]

array([67, 78,  0, 80, 70,  2,  7, 47, 64, 10, 10, 10, 32,  0, 80, 80, 63,
       47, 76,  0, 48, 35, 31, 35,  2, 41, 47,  0,  7,  2, 47,  0, 31, 31,
       47,  0, 31, 35, 66,  2, 71, 47,  2, 81,  2,  7, 63, 47, 34, 44, 78,
        0, 80, 80, 63, 47, 76,  0, 48, 35, 31, 63, 47, 35, 41, 47, 34, 44,
       78,  0, 80, 80, 63, 47, 35, 44, 47, 35, 70, 41, 47, 22, 23, 44, 10,
       23,  0, 63, 73, 10, 10, 29, 81,  2,  7, 63, 70, 78, 35, 44], dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [12]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size
       seq_length: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = batch_size * seq_length
    max_possible_batches = len(arr)//characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:max_possible_batches * characters_per_batch]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((batch_size, -1)) # (batch_size, )
        
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [10]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [11]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 8 79 82 43 81 50 59  5  7 36]
 [ 5 82 10  5 52 69 81  5 27 69]
 [40 64 52 25 36 36 74 16 50 51]
 [52  5 62 67 59 64 52 27  5 79]
 [ 5 64 81  5 64 51 73  5 51 64]
 [ 5 46 81  5 60 82 51 36 69 52]
 [79 50 52  5 29 69 10 50  5  2]
 [57  5 34 67 81  5 52 69 60  5]
 [81  5 64 51 52 54 81 25  5 53]
 [ 5 51 82 64 62  5 81 69  5 79]]

y
 [[79 82 43 81 50 59  5  7 36 36]
 [82 10  5 52 69 81  5 27 69 64]
 [64 52 25 36 36 74 16 50 51 73]
 [ 5 62 67 59 64 52 27  5 79 64]
 [64 81  5 64 51 73  5 51 64 59]
 [46 81  5 60 82 51 36 69 52 35]
 [50 52  5 29 69 10 50  5  2 69]
 [ 5 34 67 81  5 52 69 60  5 51]
 [ 5 64 51 52 54 81 25  5 53 79]
 [51 82 64 62  5 81 69  5 79 50]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [14]:
def build_inputs(batch_size, seq_length):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        seq_length: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, seq_length], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, seq_length], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [16]:
def build_lstm(num_hidden_lstm_cell, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        num_hidden_lstm_cell: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    def _build_cell(num_hidden_lstm_cell, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(num_hidden_lstm_cell)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([_build_cell(num_hidden_lstm_cell, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [17]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, same as, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape lstm_output, because it's a bunch of rows that one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output, axis=1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [31]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped =  tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [19]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [32]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, seq_length=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, seq_length = 1, 1
        else:
            batch_size, seq_length = batch_size, seq_length

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, seq_length)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [21]:
batch_size = 10         # Sequences per batch
num_steps = 50          # Number of sequence steps per batch
lstm_size = 128         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.01    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [38]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, seq_length=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state) # 이포치마다 0으로 초기화
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            print(new_state)
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
#             print('Epoch: {}/{}... '.format(e+1, epochs),
#                   'Training Step: {}... '.format(counter),
#                   'Training loss: {:.4f}... '.format(batch_loss),
#                   '{:.4f} sec/batch'.format((end-start)))
        
#             if (counter % save_every_n == 0):
#                 saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
#     saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

(LSTMStateTuple(c=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), h=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)), LSTMStateTuple(c=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), h=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  

(LSTMStateTuple(c=array([[-0.1043895 , -1.71371377, -6.03779459, ..., -2.50535965,
         0.97366881,  2.33391619],
       [-0.15290962, -1.67399454, -6.18610525, ..., -2.49462581,
         0.93334383,  2.2164793 ],
       [-0.07816795, -1.69546139, -6.19028759, ..., -2.42475653,
         1.02111363,  2.2571373 ],
       ..., 
       [-0.13523683, -1.63634849, -6.06439161, ..., -2.54103374,
         0.96608323,  2.29489827],
       [-0.11713906, -1.68065715, -6.05341721, ..., -2.56840491,
         0.93591928,  2.31716394],
       [-0.09658284, -1.65306067, -6.05024529, ..., -2.6719954 ,
         0.946594  ,  2.34155655]], dtype=float32), h=array([[-0.03945034, -0.43902504, -0.60028553, ..., -0.53246844,
         0.46853727,  0.60849553],
       [-0.06164987, -0.4516767 , -0.59414428, ..., -0.56673932,
         0.46504748,  0.61300105],
       [-0.03067993, -0.46327707, -0.61729097, ..., -0.54796094,
         0.49721956,  0.60539085],
       ..., 
       [-0.0529547 , -0.43311208, -0.

(LSTMStateTuple(c=array([[  0.09548056,  -0.7064544 , -13.89534092, ...,  -5.04195356,
          0.1713624 ,   3.13410568],
       [  0.1427338 ,  -0.70503879, -13.68564796, ...,  -4.98904896,
          0.16580138,   3.29612899],
       [  0.12751824,  -0.70809531, -13.84266472, ...,  -5.06459236,
          0.17984614,   3.27099609],
       ..., 
       [  0.11017454,  -0.71831417, -13.84801102, ...,  -5.01562786,
          0.19052225,   3.26642299],
       [  0.11347587,  -0.69815499, -13.87294483, ...,  -4.99767399,
          0.20512232,   3.27118397],
       [  0.11423746,  -0.71364492, -13.85929871, ...,  -5.01825857,
          0.19725141,   3.18226123]], dtype=float32), h=array([[ 0.02929199, -0.13853802, -0.49346146, ..., -0.45976737,
         0.04901347,  0.41476148],
       [ 0.04353678, -0.14143367, -0.52188671, ..., -0.44465399,
         0.04990336,  0.45597044],
       [ 0.03689085, -0.14432222, -0.50251609, ..., -0.43238473,
         0.05201977,  0.43345499],
       ..., 
 

(LSTMStateTuple(c=array([[  0.86073935,  -0.35804945, -11.64580441, ...,  -4.52810764,
          0.03054896,   2.44714117],
       [  0.84391749,  -0.34907562, -11.8439436 , ...,  -4.55929804,
          0.02066706,   2.48097038],
       [  0.89485884,  -0.36042303, -11.30381203, ...,  -4.62095737,
          0.03894302,   2.51085925],
       ..., 
       [  0.87304509,  -0.35215074, -11.69543552, ...,  -4.36789513,
          0.03807536,   2.38478589],
       [  0.9114905 ,  -0.35363936, -11.40907097, ...,  -4.46922493,
          0.03474669,   2.53448772],
       [  0.86374891,  -0.34817857, -11.59432793, ...,  -4.44543123,
          0.03244831,   2.47825146]], dtype=float32), h=array([[ 0.35771716, -0.05818023, -0.40268737, ..., -0.27636307,
         0.00567826,  0.32727677],
       [ 0.36299288, -0.05744493, -0.39517936, ..., -0.28751799,
         0.00435545,  0.31775841],
       [ 0.3747555 , -0.05702845, -0.39493331, ..., -0.27698144,
         0.00742968,  0.32187247],
       ..., 
 

(LSTMStateTuple(c=array([[  5.21948636e-01,  -2.99773574e-01,  -9.25543976e+00, ...,
         -3.92636013e+00,  -3.93941905e-03,   1.64284074e+00],
       [  4.98151422e-01,  -3.04830760e-01,  -9.41205025e+00, ...,
         -3.98413038e+00,   1.24450019e-02,   1.66686559e+00],
       [  4.96438444e-01,  -2.99112439e-01,  -9.44966888e+00, ...,
         -3.81729174e+00,   2.33509485e-03,   1.57380414e+00],
       ..., 
       [  5.20755768e-01,  -2.91811168e-01,  -9.45886326e+00, ...,
         -3.99778032e+00,  -6.96533825e-03,   1.63230479e+00],
       [  5.59567213e-01,  -3.02486986e-01,  -9.26061249e+00, ...,
         -4.01014948e+00,   1.94235817e-02,   1.61779976e+00],
       [  5.68362355e-01,  -3.10077131e-01,  -9.47971058e+00, ...,
         -3.94882631e+00,   1.15542039e-02,   1.60941982e+00]], dtype=float32), h=array([[ 0.21554904, -0.05120463, -0.33058938, ..., -0.31263363,
        -0.00063443,  0.23037414],
       [ 0.19313818, -0.0455813 , -0.33938804, ..., -0.27519768,
     

...

(LSTMStateTuple(c=array([[ 0.54055911, -0.16292176, -5.5340991 , ..., -1.36920214,
         0.02288678,  0.16797519],
       [ 0.30034262, -0.10677384, -5.22144032, ..., -1.34148943,
         0.00935897,  0.10873602],
       [ 0.26960686, -0.2134977 , -5.67613792, ..., -1.61696911,
         0.08108735,  0.35152024],
       ..., 
       [ 0.34210038, -0.12028778, -5.10852242, ..., -1.33024132,
         0.01099924,  0.12613069],
       [ 0.06232154, -0.15188158, -5.2430439 , ..., -1.51122522,
         0.03626825,  0.18063536],
       [ 0.27236941, -0.23772867, -5.35602188, ..., -1.38918555,
         0.01349378,  0.17115241]], dtype=float32), h=array([[ 0.4185251 , -0.02186411, -0.200596  , ..., -0.22865097,
         0.00291483,  0.01390057],
       [ 0.21997145, -0.02121757, -0.2417068 , ..., -0.3513191 ,
         0.00197234,  0.00989765],
       [ 0.13500828, -0.02294037, -0.25478366, ..., -0.34993735,
         0.01441323,  0.05094837],
       ..., 
       [ 0.25161985, -0.02177355, -0.

(LSTMStateTuple(c=array([[ 0.57969385, -0.18917109, -5.83360863, ..., -1.8148582 ,
         0.07997113,  0.56261301],
       [-0.05135966, -0.2031575 , -5.25722265, ..., -1.31645644,
         0.10039444,  0.27568823],
       [ 0.16988695, -0.15267593, -5.72570515, ..., -1.11994815,
         0.02703661,  0.19215859],
       ..., 
       [ 0.00598639, -0.19502726, -5.3538065 , ..., -1.28914022,
         0.02377134,  0.20096487],
       [ 0.43581444, -0.19383219, -5.5923481 , ..., -1.81162477,
         0.06842201,  0.49888384],
       [ 0.28774002, -0.15214856, -5.23910618, ..., -1.59369206,
         0.10771218,  0.26850352]], dtype=float32), h=array([[ 0.45214945, -0.02547502, -0.46314284, ..., -0.47379279,
         0.0133014 ,  0.11077102],
       [-0.03254109, -0.04227383, -0.21792695, ..., -0.18932931,
         0.02384652,  0.0575478 ],
       [ 0.12898116, -0.02863809, -0.15829942, ..., -0.16347635,
         0.00497258,  0.02713996],
       ..., 
       [ 0.00440632, -0.05223714, -0.

(LSTMStateTuple(c=array([[-0.13042322, -0.19549939, -5.24004936, ..., -1.05004621,
         0.13551566,  0.17173716],
       [ 0.0289798 , -0.09067001, -5.57171059, ..., -1.26305318,
         0.01506625,  0.08130074],
       [ 0.57592452, -0.11234651, -5.35008717, ..., -0.91417289,
         0.01808295,  0.12895533],
       ..., 
       [ 0.14445226, -0.07106856, -5.2175746 , ..., -1.42583084,
        -0.01822473,  0.08042814],
       [-0.17353779, -0.07403658, -5.00672722, ..., -0.98183751,
         0.03692894,  0.07083665],
       [ 0.29868245, -0.1437398 , -5.0251708 , ..., -1.72372079,
         0.10728094,  0.31689855]], dtype=float32), h=array([[-0.09558054, -0.04532013, -0.1804684 , ..., -0.14799438,
         0.03636672,  0.03015274],
       [ 0.02708721, -0.00877504, -0.13337687, ..., -0.3733924 ,
         0.00189982,  0.00258596],
       [ 0.49003884, -0.01341615, -0.20353335, ..., -0.1651237 ,
         0.00237738,  0.00743479],
       ..., 
       [ 0.13799393, -0.00711011, -0.

(LSTMStateTuple(c=array([[  1.05879702e-01,  -2.68638939e-01,  -5.18932295e+00, ...,
         -1.20827436e+00,   7.87846893e-02,   2.49007374e-01],
       [  1.29909560e-01,  -8.39443654e-02,  -4.92085934e+00, ...,
         -1.39138293e+00,   6.39394447e-02,   1.74322188e-01],
       [  1.51142970e-01,  -1.87344983e-01,  -5.09785700e+00, ...,
         -1.56990957e+00,   1.23072192e-01,   4.03912842e-01],
       ..., 
       [ -2.88832754e-01,  -1.10857211e-01,  -5.03489447e+00, ...,
         -1.61378932e+00,   4.43250593e-03,   6.46278262e-02],
       [ -1.42446548e-01,  -1.75075471e-01,  -5.23559999e+00, ...,
         -1.43470144e+00,   4.19062935e-02,   1.10302135e-01],
       [ -2.38420457e-01,  -2.19371289e-01,  -5.01407146e+00, ...,
         -1.07092094e+00,   9.53629613e-02,   1.42391816e-01]], dtype=float32), h=array([[ 0.06754848, -0.06773748, -0.1818822 , ..., -0.22206677,
         0.02059337,  0.03650852],
       [ 0.11861712, -0.00911306, -0.33906347, ..., -0.32564011,
     

KeyboardInterrupt: 

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [ ]:
tf.train.get_checkpoint_state('checkpoints')

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [ ]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [ ]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [ ]:
tf.train.latest_checkpoint('checkpoints')

In [ ]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

In [ ]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

In [ ]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

In [ ]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)